# BatchNorm和LayerNorm

🚩目录
    ①二维数据的BatchNorm和LayerNorm
    ②图像数据的BatchNorm和LayerNorm
    ③文本数据的BatchNorm和LayerNorm

   ###  ① 二维数据的BatchNorm和LayerNorm
   假设有 **二维数据 X**:
   Batchsize为3，Feature个数为4（每个样本由四个特征构成）

In [7]:
import torch

X = torch.arange(12, dtype=torch.float).reshape(3, 4)
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11.]])

看看概念：

   BatchNorm是对这个batch数据中的**每个特征**进行样本间的归一化（也就是上面的X进行纵向的归一化
   
   LayerNorm是对**每个样本**在其特征维度上进行归一化（上面的X进行横向归一化
   
操作和结果如下：(注意观察mean和var的形状)

In [22]:
# 求均值和方差
batch_mean = torch.mean(X, dim=0, keepdim=True)
batch_var = torch.var(X, dim=0, keepdim=True)  # 计算样本方差除以n-1而不是n
# 计算
batch_X = (X - batch_mean)/torch.sqrt(batch_var)
batch_mean, batch_var, batch_X

(tensor([[4., 5., 6., 7.]]),
 tensor([[16., 16., 16., 16.]]),
 tensor([[-1., -1., -1., -1.],
         [ 0.,  0.,  0.,  0.],
         [ 1.,  1.,  1.,  1.]]))

In [21]:
# 求均值和方差
layer_mean = torch.mean(X, dim=1, keepdim=True)
layer_var = torch.var(X, dim=1, keepdim=True)
# 计算
layer_X = (X - layer_mean)/torch.sqrt(layer_var)
layer_mean, layer_var, layer_X

(tensor([[1.5000],
         [5.5000],
         [9.5000]]),
 tensor([[1.6667],
         [1.6667],
         [1.6667]]),
 tensor([[-1.1619, -0.3873,  0.3873,  1.1619],
         [-1.1619, -0.3873,  0.3873,  1.1619],
         [-1.1619, -0.3873,  0.3873,  1.1619]]))

 ###  ② 图像数据的BatchNorm和LayerNorm

图像数据有四个维度 (B, C, H, W),分别是 batch大小，通道数量，高， 宽

In [29]:
# X表示一个batch中的两张三通道图片，图片的高和宽都为两个像素
X = torch.arange(24, dtype=torch.float).reshape(2, 3, 2, 2)
X.shape, X

(torch.Size([2, 3, 2, 2]),
 tensor([[[[ 0.,  1.],
           [ 2.,  3.]],
 
          [[ 4.,  5.],
           [ 6.,  7.]],
 
          [[ 8.,  9.],
           [10., 11.]]],
 
 
         [[[12., 13.],
           [14., 15.]],
 
          [[16., 17.],
           [18., 19.]],
 
          [[20., 21.],
           [22., 23.]]]]))

BatchNorm和LayerNorm的定义还是不变的，重点在找到特征维度

如一张RGB图片由红绿蓝三个通道组成，那么每一个通道就是这个图片的一个特征，只不过每一个特征都是二维张量

所以C这一个维度就是特征维度

#### 2.1 图片的BatchNorm
BatchNorm的方向类似于：

<img src="..\Apictures\2.jpg" alt="2" style="zoom:15%;" />

所以生成的mean和var的数量和C也就是特征的数量是一样的，现在你再回去读下BatchNorm的定义，会有更直观的理解。

下面我们用pytorch中自带的类来演示图片的BatchNorm操作：

In [30]:
# 这里我们直接调用pytorch中的BatchNorm2d来进行处理
bn2d = torch.nn.BatchNorm2d(num_features=3)
bn2d

BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

<img src="..\Apictures\1.png" alt="1" style="zoom:50%;" />
这个公式便是BatchNorm2d所做的事情

除了归一化，还有γ和β可以对标准正态分布进行偏移（变换方差和均值），初始化分别为1和0。这两个参数是可以学习的，最终是多少由训练过程决定，而不是我们指定。当然我们也不是什么都不能做，我们可以通过在实例化对象时指定affine=False让这两个参数变成不可学习的，那么它俩就固定为1和0了。

In [35]:
# weight和bias就是指γ和β，它俩的长度和mean,var的对数(这个对数不是指log!)相同，也就是和C相同，
# 所以实例化bn2d对象时要传入C，进行weight和bias的初始化
bn2d.weight, bn2d.bias

(Parameter containing:
 tensor([1., 1., 1.], requires_grad=True),
 Parameter containing:
 tensor([0., 0., 0.], requires_grad=True))

参数解释：
- num_features:特征/通道的数量，因为batchnorm实例化需要知道γ和β的形状，他俩的形状和C一样，所以在实例化该对象的时候需要传入C。
- momentum=0.1:默认0.1，用于学习过程中更新γ和β，具体更新公式如下：
<img src="..\Apictures\3.png" alt="3" style="zoom:25%;" />
- affine=True:控制γ和β是否可以学习，默认可以学习
- eps:公式中的ε，防止分母为0
- tracking_running_stats:控制是否要更新running_mean和running_var,这两个变量用作模型eval的mean和var,如果tracking_running_stats设置为false，那么这两个变量都为None，模型eval时直接根据数据计算mean和var

In [31]:
batchnorm_X = bn2d(X)
batchnorm_X

tensor([[[[-1.2288, -1.0650],
          [-0.9012, -0.7373]],

         [[-1.2288, -1.0650],
          [-0.9012, -0.7373]],

         [[-1.2288, -1.0650],
          [-0.9012, -0.7373]]],


        [[[ 0.7373,  0.9012],
          [ 1.0650,  1.2288]],

         [[ 0.7373,  0.9012],
          [ 1.0650,  1.2288]],

         [[ 0.7373,  0.9012],
          [ 1.0650,  1.2288]]]], grad_fn=<NativeBatchNormBackward0>)

用上面数据进行验证：

取两个样本的第一个特征[-1.2288, -1.0650, -0.9012, -0.7373]和[ 0.7373,  0.9012, 1.0650,  1.2288],相加的确等于0，说明服从标准正太分布

其余特征这里就不验证了，可自行相加验证。

#### 2.2 图片的LayerNorm
LayerNorm的方向类似于：
<img src="..\Apictures\7.jpg" alt="7" style="zoom:15%;" />

In [40]:
# 这里我们直接调用pytorch中的LayerNorm来进行处理
layer_norm = torch.nn.LayerNorm(normalized_shape=[3, 2, 2])
layer_norm, layer_norm.weight.shape, layer_norm.bias.shape

(LayerNorm((3, 2, 2), eps=1e-05, elementwise_affine=True),
 torch.Size([3, 2, 2]),
 torch.Size([3, 2, 2]))

需要说明的是，虽然对每一个样本计算一对mean和var（在这个例子中总共两对），但是layerNorm的γ和β却不止有两个，而是element-wise的。

具体来说，对于一个最基本的像素点和它在其它样本中相同位置的所有像素点，分配一对γ和β。是这样的，但是为什么要这样设计，就要问layernorm的提出者了。

跟batchnorm相同，layernorm实例化的时候要初始化weights和bias,也就是γ和β的形状，所以需要传入normalized_shape

<img src="..\Apictures\6.png" alt="6" style="zoom:50%;" />


In [43]:
layernorm_X = layer_norm(X)
layernorm_X

tensor([[[[-1.5933, -1.3036],
          [-1.0139, -0.7242]],

         [[-0.4345, -0.1448],
          [ 0.1448,  0.4345]],

         [[ 0.7242,  1.0139],
          [ 1.3036,  1.5933]]],


        [[[-1.5933, -1.3036],
          [-1.0139, -0.7242]],

         [[-0.4345, -0.1448],
          [ 0.1448,  0.4345]],

         [[ 0.7242,  1.0139],
          [ 1.3036,  1.5933]]]], grad_fn=<NativeLayerNormBackward0>)

 ###  ③  文本数据的BatchNorm和LayerNorm

在transformer及rnn这些语言模型中处理的文本向量一般形状为：     (B, T, H)

其中，
- B是batch_size，也就是这个batch中有几句话
- T可以理解为每句话中的单词数量
- H可以理解为每一个单词被表示为几维的向量


In [46]:
X = torch.randn([2, 3, 5])
X # 每一行可以看作一个字，每三行表示一个句子

tensor([[[-1.0364,  0.4676, -0.6561, -0.7976, -0.0697],
         [ 0.1216,  0.9560,  1.5325,  0.0071, -1.0951],
         [-0.0868,  0.4512,  0.1544,  0.3392,  0.5351]],

        [[-0.2735,  1.5836, -0.6444,  0.2002, -0.9999],
         [ 0.6813,  1.1774, -1.6905,  1.8488,  0.3426],
         [ 0.8175,  0.0263, -0.7140, -1.4588,  0.0625]]])

transformer中用的是layernorm，所以现在大多数处理文本的模型用的都是layernorm，所以这里就只演示layernorm

（其实是对batchnorm1d有些魔迷惑，所以就不误导读者了哈哈哈哈）

这里用的layerNorm跟图片用的layernorm是一摸一样的，只是传入的normalized_shape不一样

In [55]:
layer_norm = torch.nn.LayerNorm(normalized_shape=5)
layer_norm, layer_norm.weight.shape, layer_norm.bias.shape

(LayerNorm((5,), eps=1e-05, elementwise_affine=True),
 torch.Size([5]),
 torch.Size([5]))

In [56]:
layernorm_X = layer_norm(X)
layernorm_X

tensor([[[-1.1325,  1.6237, -0.4355, -0.6949,  0.6391],
         [-0.2041,  0.7273,  1.3708, -0.3319, -1.5621],
         [-1.6399,  0.7746, -0.5575,  0.2717,  1.1511]],

        [[-0.2746,  1.7928, -0.6875,  0.2527, -1.0834],
         [ 0.1753,  0.5908, -1.8108,  1.1530, -0.1083],
         [ 1.3847,  0.3616, -0.5957, -1.5589,  0.4083]]],
       grad_fn=<NativeLayerNormBackward0>)

In [57]:
summ = torch.sum(layernorm_X,dim=2)
summ 

tensor([[-2.3842e-07, -2.9802e-08, -1.1921e-07],
        [-2.9802e-07, -2.3842e-07,  0.0000e+00]], grad_fn=<SumBackward1>)

如上，由于浮点数经度原因，相加不能完全为0，但是从e-08可以看出基本就是0了